#### The computer vision project uses machine learning, radar technology, and OCR to analyze Egyptian license plates 🚘. EasyOCR library is employed for Arabic OCR. The code processes images, detects plate edges, and uses OCR for text. The results are displayed visually, demonstrating the integration of computer vision and OCR for license plate analysis. 🚘🚘

## 1 - Import librarys necessary to done the project 

In [ ]:
import os
from easyocr import Reader
import cv2
import matplotlib.pylab as plt

## 2 - Reading the data set file

In [ ]:
image_folder = 'C:/Users/alabdeen/Desktop/project cv/data3'

### 3 - This line prepares an OCR reader in the Arabic programming language using a central processing unit (CPU) and without printing details of internal operations.

In [ ]:
reader = Reader(['ar'], gpu=False, verbose=False)

In [ ]:
# Image upload and processing:

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_folder, filename)
        car = cv2.imread(image_path)
        car = cv2.resize(car, (800, 600))

        # Image processing for edge detection
        """
        The image is converted to grayscale and smoothed using a Gaussian Blur filter. 
        Canny technology is then used to detect the edges of the board.
        """
        gray = cv2.cvtColor(car, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blur, 10, 200)

        # Identify the panel:
        """
        The Contours technique is used to identify edges, and the five largest edges are selected.
        The plate is defined by searching for approximate polygons after finding the edge.
        """
        cont, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cont = sorted(cont, key=cv2.contourArea, reverse=True)[:5]
        plate_cnt = None
        for c in cont:
            arc = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * arc, True)
            if len(approx) == 4:
                plate_cnt = approx
                break

        # Read text from panel using OCR:
        """
        If the plate is found, OCR is used to read the text.
        If no text is readable, a message indicating that the text is difficult to read is displayed on the panel.
        If the text is read successfully, it is displayed on the image with the panel selected.
        """
        if plate_cnt is not None:
            (x, y, w, h) = cv2.boundingRect(plate_cnt)
            x = max(0, x)
            y = max(0, y)
            plate_roi = gray[y:y + h, x:x + w]

            detection = reader.readtext(plate_roi)

            if len(detection) == 0:
                text = "Impossible to read the text from the license plate"
                cv2.putText(car, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)
            else:
                cv2.drawContours(car, [plate_cnt], -1, (255, 0, 0), 3)
                text = f"{detection[0][1]} {detection[0][2] * 100:.2f}%"
                cv2.putText(car, text, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
                print(text)

        # Display the original image
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(car, cv2.COLOR_BGR2RGB))
        plt.title("Image")

        # If the board is found, display a thumbnail of the board
        if plate_cnt is not None:
            plt.subplot(1, 3, 2)
            plt.imshow(cv2.cvtColor(plate_roi, cv2.COLOR_BGR2RGB))
            plt.title("License Plate")

            # Display the original image with the panel selected
            plt.subplot(1, 3, 3)
            plt.imshow(cv2.cvtColor(car, cv2.COLOR_BGR2RGB))
            plt.title("Image with Contours")

        plt.show()
